Анализ текста

Импортируем все необходимое
import re: Импортирует модуль для работы с регулярными выражениями, который позволяет выполнять сложные текстовые манипуляции.
from collections import Counter: Импортирует класс Counter из модуля collections, который используется для подсчета объектов (например, слов) в списке.
import spacy: Импортирует библиотеку Spacy, которая предназначена для обработки естественного языка.
from Levenshtein import ratio: Импортирует функцию ratio из библиотеки Levenshtein для вычисления расстояния Левенштейна, которое измеряет схожесть между двумя строками.
from spacy.lang.en.stop_words import STOP_WORDS: Импортирует предопределенный список стоп-слов из Spacy, которые часто не несут значимого смысла в тексте (например, "и", "но", "или").
from string import punctuation: Импортирует строку, содержащую все знаки препинания (например, ".", ",", "!").

In [ ]:
import re
from collections import Counter
import spacy
from Levenshtein import ratio
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

Функция read_file принимает путь к файлу (file_path) и открывает файл в режиме чтения с кодировкой UTF-8.
Читает содержимое файла и возвращает его как строку.
Функция для чтения файла

In [ ]:
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

Функция clean_text очищает текст, приводя его к нижнему регистру.
Удаляет все знаки препинания с помощью регулярного выражения.
Разбивает текст на отдельные слова.
Исключает стоп-слова из списка слов.
Возвращает очищенный текст в виде строки.
Функция для очистки текста от стоп-слов и знаков препинания

In [ ]:
def clean_text(text):
    text = text.lower()  #Приведение текста к нижнему регистру
    text = re.sub(f'[{re.escape(punctuation)}]', '', text)  #Удаление знаков препинания
    words = text.split()
    words = [word for word in words if word not in STOP_WORDS]  #Исключение стоп-слов
    return ' '.join(words)

Функция count_unique_words разбивает текст на слова.
Преобразует список слов в множество, чтобы удалить дубликаты.
Возвращает количество уникальных слов.
Функция для подсчета уникальных слов

In [ ]:
def count_unique_words(text):
    words = text.split()
    unique_words = set(words)
    return len(unique_words)

Функция count_vowels_consonants определяет списки гласных и согласных букв.
Использует генераторы списка для подсчета гласных и согласных в тексте.
Возвращает количество гласных и согласных букв.
Функция для подсчета гласных и согласных

In [ ]:
def count_vowels_consonants(text):
    vowels = "aeiou"
    consonants = "bcdfghjklmnpqrstvwxyz"
    
    vowels_count = sum(1 for char in text if char in vowels)
    consonants_count = sum(1 for char in text if char in consonants)
    
    return vowels_count, consonants_count

Функция analyze_sentences_words загружает небольшую модель Spacy.
Применяет модель к тексту, чтобы разметить предложения и слова.
Извлекает предложения и подсчитывает их количество.
Вычисляет длину каждого предложения.
Извлекает слова и подсчитывает их частоту.
Функция для подсчета предложений, их длины и частоты слов с критерием схожести

In [ ]:
def analyze_sentences_words(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    
    sentences = list(doc.sents)
    num_sentences = len(sentences)
    
    sentence_lengths = [len(sent.text.split()) for sent in sentences]
    
    words = [token.text.lower() for token in doc if token.is_alpha]
    word_freq = Counter(words)
    
    return num_sentences, sentence_lengths, word_freq

Функция process_words_with_similarity итерирует по списку слов и проверяет их схожесть с уникальными словами, используя расстояние Левенштейна.
Если слово достаточно похоже на уже встреченное слово, увеличивает его счетчик частоты.
Если слово уникально, добавляет его в список уникальных слов и увеличивает его счетчик.
Функция для обработки слов с использованием расстояния Левенштейна

In [ ]:
def process_words_with_similarity(words, threshold=0.8):
    unique_words = []
    word_freq = Counter()
    
    for word in words:
        is_unique = True
        for uw in unique_words:
            if ratio(word, uw) > threshold:
                word_freq[uw] += 1
                is_unique = False
                break
        if is_unique:
            unique_words.append(word)
            word_freq[word] += 1
    
    return word_freq

Функция для вывода топ-10 часто встречаемых слов

In [ ]:
def get_top_words(word_freq, top_n=10):
    return word_freq.most_common(top_n)

Основная программа

In [ ]:
file_path = r'C:\Users\1neon\Desktop\fightclub.txt'  # Указанный путь к файлу
text = read_file(file_path)

Очистка текста

In [ ]:
cleaned_text = clean_text(text)

Подсчет уникальных слов

In [ ]:
unique_words_count = count_unique_words(cleaned_text)
print(f"Число уникальных слов: {unique_words_count}")

Подсчет гласных и согласных

In [ ]:
vowels_count, consonants_count = count_vowels_consonants(cleaned_text)
print(f"Число гласных: {vowels_count}")
print(f"Число согласных: {consonants_count}")

Анализ предложений и слов

In [ ]:
num_sentences, sentence_lengths, word_freq = analyze_sentences_words(cleaned_text)
print(f"Число предложений: {num_sentences}")
print(f"Длины предложений: {sentence_lengths}")

Обработка слов с критерием схожести

In [ ]:
words = cleaned_text.split()
similarity_word_freq = process_words_with_similarity(words)

Топ-10 часто встречаемых слов

In [ ]:
top_words = get_top_words(similarity_word_freq)
print("\nТоп-10 часто встречаемых слов:")
for word, freq in top_words:
    print(f"'{word}': {freq} раз")


Вывод: 
Число уникальных слов: 5127
Число гласных: 44254
Число согласных: 82120
Число предложений: 218
Длины предложений: [519, 83, 82, 50, 373, 237, 111, 65, 9, 1, 2, 7, 2, 2, 1, 7, 74, 7, 7, 86, 77, 99, 196, 39, 514, 44, 212, 113, 150, 1, 46, 186, 86, 11, 16, 162, 229, 134, 4, 147, 146, 89, 88, 5, 297, 200, 231, 361, 55, 28, 282, 405, 5, 27, 5, 111, 275, 33, 211, 1, 1, 1, 1, 309, 94, 244, 2, 1, 1, 324, 21, 8, 2, 2, 2, 116, 139, 137, 148, 151, 95, 222, 112, 477, 65, 34, 23, 29, 115, 80, 276, 174, 174, 41, 9, 50, 4, 24, 121, 56, 727, 3, 108, 359, 11, 145, 238, 949, 94, 163, 154, 27, 221, 21, 90, 2, 7, 37, 226, 59, 98, 1, 1, 1, 1, 1, 1, 231, 9, 298, 19, 95, 11, 57, 174, 654, 30, 36, 150, 272, 333, 29, 161, 17, 110, 95, 97, 1, 1, 1, 6, 2, 43, 26, 1, 10, 65, 201, 44, 147, 167, 329, 111, 140, 100, 26, 1, 1, 34, 107, 4, 15, 2, 431, 44, 401, 22, 213, 120, 33, 15, 1, 22, 177, 24, 240, 236, 14, 1, 1, 1, 9, 1, 3, 48, 20, 31, 136, 71, 49, 243, 1, 1, 1, 173, 15, 66, 91, 19, 46, 587, 59, 93, 15, 1, 27, 2, 42]

Топ-10 часто встречаемых слов:
'tyler': 677 раз
'marla': 342 раз
'says': 326 раз
'fight': 246 раз
'club': 185 раз
'dont': 147 раз
'said': 138 раз
'im': 135 раз
'guy': 134 раз
'know': 130 раз
PS C:\Users\1neon>